In [ ]:
#|default_exp cnn_virus.architecture

In [ ]:
#|hide
import nbdev

from eccore.ipython import nb_setup
from fastcore.test import *
from metagentorch.cnn_virus.utils import setup_nb
from nbdev.showdoc import *

In [ ]:
#| hide
nb_setup()
ON_COLAB, p2dataroot, p2data = setup_nb(_dev=True)

Set autoreload mode
Running locally


In [ ]:
#|export
# Imports all dependencies
import os
import torch

from pathlib import Path
from typing import Callable, Tuple
from metagentorch.core import ProjectFileSystem

In [ ]:
#|export
# Set pytorch as backend
os.environ['KERAS_BACKEND'] = 'torch'

In [ ]:
#| export
import keras
from keras.layers import Convolution1D, Dense, Flatten, Dropout, Activation, BatchNormalization, Input
from keras.layers import MaxPooling1D, Concatenate
from keras.models import Sequential, Model, load_model

In [ ]:
#|hide
print(f"PyTorch version: {torch.__version__} - Expected 2.5.1")
print(f"Keras version: {keras.__version__} - Expected 3.8.0")

PyTorch version: 2.5.1 - Expected 2.5.1
Keras version: 3.8.0 - Expected 3.8.0


# architecture

> DL architectures based on CNN Virus and used in this project

The paper *"A multi-task CNN learning model for taxonomic assignment of human viruses"* proposes a CNN model applied to virus. 

> "A multi-task CNN learning model for taxonomic assignment of human viruses"
>
> by Haoran Ma, Tin Wee Tan and Kenneth Hon Kim Ban
>
> From 19th International Conference on Bioinformatics 2020 (InCoB2020) - Virtual. 25-29 November 2020

The module `architecture` includes:

- The architecture from used in the original paper model, refactor to use the latest version of Keras. This is based on the authors' [github repo](https://github.com/MaHaoran627/CNN_Virus).
- All other variants we are developing, whether they have the same architecture and are simply retrained or finetuned, or whether they adopt a new architecture.

## Original Architecture

Below are the functions to create model with the same architecture as the original paper.

In [ ]:
#| export

def create_model_original(
    load_parameters: bool = True, # Load pretrained weights when True
    path2parameters: Path|None = None,  # Path to pretrained weights, defaults to project CNN Virus weights
    ) -> keras.Model:          # New instance of an original paper architecture
    """Build a CNN model as per CNN Virus paper"""

    print("Creating CNN Model (Original)")
    # Build cnn model
    input_seq=Input(shape=(50,5), name='input-seq')
    layer1=Convolution1D(512, 5, padding="same",activation="relu",kernel_initializer="he_uniform", name="conv-1")(input_seq)
    layer2=BatchNormalization(momentum=0.6, name='bn-1')(layer1)
    layer3=MaxPooling1D(pool_size=2,padding='same', name='maxpool-1')(layer2)
    layer4=Convolution1D(512, 5, padding="same",activation="relu",kernel_initializer="he_uniform", name="conv-2")(layer3)
    layer5=BatchNormalization(momentum=0.6, name='bn-2')(layer4)
    layer6=MaxPooling1D(pool_size=2,padding='same',name='maxpool-2')(layer5)
    layer7=Convolution1D(1024, 7, padding="same", activation="relu",kernel_initializer="he_uniform", name="conv-3")(layer6)
    layer8=Convolution1D(1024, 7, padding="same", activation="relu",kernel_initializer="he_uniform", name="conv-4")(layer7)
    layer9=BatchNormalization(momentum=0.6, name='bn-3')(layer8)
    layer10=MaxPooling1D(pool_size=2,padding='same', name='maxpool-3')(layer9)
    layer11=Flatten(name='flatten')(layer10)
    layer12=Dense(1024,kernel_initializer="he_uniform", name='dense-1')(layer11)
    layer13=BatchNormalization(momentum=0.6, name='bn-4')(layer12)
    layer14=Dropout(0.2, name='do-1')(layer13)
    labels=Dense(187, activation='softmax',kernel_initializer="he_uniform",name="labels")(layer14)
    output_con=Concatenate(name='concat')([layer14,labels])
    layer15=Dense(1024, kernel_initializer="he_uniform", name='dense-2')(output_con)
    layer16=BatchNormalization(momentum=0.6, name='bn-5')(layer15)
    pos=Dense(10, activation='softmax',kernel_initializer="he_uniform",name="pos")(layer16)
    model = Model(inputs=input_seq, outputs=[labels,pos], name="CNN_Virus")

    # Load pretrained weights
    if load_parameters:
        if path2parameters is None: 
            path2parameters = ProjectFileSystem().data /'saved/cnn_virus_original/pretrained_model.h5'
        if not path2parameters.is_file(): 
            raise FileNotFoundError(f"Could not find pretrained model at {path2parameters}")
        print(f"Loading parameters from {path2parameters.name}")
        model.load_weights(path2parameters)
        print("Created pretrained model")
    else:
        print("Created randomly initialized model")
    return model    

In [ ]:
ProjectFileSystem().data

Path('/home/vtec/projects/bio/metagentorch/data')

This is the original paper's model, taking 50-mer sequences and predicting:

1. the logits for 187 viruses
2. the logits for one of 10 regions in the original virus genome

In [ ]:
model = create_model_original(load_parameters=False)

Creating CNN Model (Original)
Created randomly initialized model


In [ ]:
model.summary()

Model: "CNN_Virus"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input-seq           │ (None, 50, 5)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv-1 (Conv1D)     │ (None, 50, 512)   │     13,312 │ input-seq[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn-1                │ (None, 50, 512)   │      2,048 │ conv-1[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool-1           │ (None, 25, 512)   │          0 │ bn-1[0][0]        │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv-2 (Conv1D)     │ (None, 25, 512)   │  1,311,232 │ maxpool-1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn-2                │ (None, 25, 512)   │      2,048 │ conv-2[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool-2           │ (None, 13, 512)   │          0 │ bn-2[0][0]        │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv-3 (Conv1D)     │ (None, 13, 1024)  │  3,671,040 │ maxpool-2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv-4 (Conv1D)     │ (None, 13, 1024)  │  7,341,056 │ conv-3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn-3                │ (None, 13, 1024)  │      4,096 │ conv-4[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool-3           │ (None, 7, 1024)   │          0 │ bn-3[0][0]        │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 7168)      │          0 │ maxpool-3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense-1 (Dense)     │ (None, 1024)      │  7,341,056 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn-4                │ (None, 1024)      │      4,096 │ dense-1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ do-1 (Dropout)      │ (None, 1024)      │          0 │ bn-4[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ labels (Dense)      │ (None, 187)       │    191,675 │ do-1[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat              │ (None, 1211)      │          0 │ do-1[0][0],       │
│ (Concatenate)       │                   │            │ labels[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense-2 (Dense)     │ (None, 1024)      │  1,241,088 │ concat[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn-5                │ (None, 1024)      │      4,096 │ dense-2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pos (Dense)         │ (None, 10)        │     10,250 │ bn-5[0][0]      

 Total params: 21,137,093 (80.63 MB)

 Trainable params: 21,128,901 (80.60 MB)

 Non-trainable params: 8,192 (32.00 KB)

In [ ]:
#|eval: false
model = create_model_original(load_parameters=True)

Creating CNN Model (Original)
Loading parameters from pretrained_model.h5
Created pretrained model


In [ ]:
#| hide
nbdev.nbdev_export()